In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Data Preprocessing

In [2]:
data = pd.read_csv('Earthquate_Damage.csv') 
data.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,3
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,2
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,3
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,2
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,3


In [3]:
def data_explanation(data, name):
    """
    create a txt file that contains the explanation of the data
    """
    
    with open(f'./data_description_{name}.txt','w') as f:
        for i in data.columns:
            f.write(f'Feature Name: {i} \n')
            f.write(f'# of data: {len(data[i])} \n')
            f.write(f'# of unique data: {len(data[i].unique())} \n')
            f.write(f'unique datas: {data[i].unique()} \n\n')
    f.close()
    
data_explanation(data, 'earthquake')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 40 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   building_id                             260601 non-null  int64 
 1   geo_level_1_id                          260601 non-null  int64 
 2   geo_level_2_id                          260601 non-null  int64 
 3   geo_level_3_id                          260601 non-null  int64 
 4   count_floors_pre_eq                     260601 non-null  int64 
 5   age                                     260601 non-null  int64 
 6   area_percentage                         260601 non-null  int64 
 7   height_percentage                       260601 non-null  int64 
 8   land_surface_condition                  260601 non-null  object
 9   foundation_type                         260601 non-null  object
 10  roof_type                               260601 non-null 

Data Description file 참조 결과,

- building ID는 unique data 개수가 총 데이터 수와 같아, 제거 되어야함. Unnecessary Identifiers

- info() function 결과를 보면 모든 columnn에 null 값이 없음을 볼 수 있음.

[1] 입력 변수의 속성이 numeric 이 아닌 변수들에 대해 1-of-C coding (1-hot encoding) 방식을 통해 
명목형(요인형) 변수를 범주의 개수만큼의 이진형(binary)  변수들로 구성되는 dummy  variable  을 
생성하시오.

In [5]:
features = data.drop(['building_id','damage_grade'], axis = 1)
target = data['damage_grade']

In [6]:
#Object type features list
OHE_features = list(features.select_dtypes(include = ['object']).columns)
len(OHE_features)

8

In [7]:
OHE_features

['land_surface_condition',
 'foundation_type',
 'roof_type',
 'ground_floor_type',
 'other_floor_type',
 'position',
 'plan_configuration',
 'legal_ownership_status']

In [8]:
# one hot encoding
OHE = OneHotEncoder(drop='first')
One_hot_encoded = OHE.fit_transform(features[OHE_features])
One_hot_encoded_dt = pd.DataFrame(One_hot_encoded.toarray(), columns = OHE.get_feature_names_out(OHE_features))

features = features.drop(OHE_features, axis = 1)
features = pd.concat([features, One_hot_encoded_dt], axis = 1)

features.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
0,6,487,12198,2,30,6,5,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,8,900,2812,2,10,8,7,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,21,363,8973,2,10,5,5,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,22,418,10694,2,10,6,5,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,11,131,1488,3,30,8,9,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# scale the numerical features   
numerical_columns = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage', 'count_families']

scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(features[numerical_columns])
scaled_numerical = pd.DataFrame(scaled_numerical, columns = numerical_columns)

features = features.drop(numerical_columns, axis = 1)
features = pd.concat([scaled_numerical, features], axis = 1)

features.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,count_families,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,...,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
0,-0.983414,-0.518705,1.629055,-0.178274,0.047100,-0.459460,-0.226419,0.038365,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.734459,0.481998,-0.945017,-0.178274,-0.224765,-0.004110,0.816109,0.038365,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.883744,-0.819158,0.744612,-0.178274,-0.224765,-0.687135,-0.226419,0.038365,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.008221,-0.685893,1.216589,-0.178274,-0.224765,-0.459460,-0.226419,0.038365,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.361028,-1.381296,-1.308119,1.195989,0.047100,-0.004110,1.858636,0.038365,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


[2] 전체 데이터셋을 임의로 150,000 개의 빌딩이 포함된 Training dataset 과 50,000 개의 Validation 
dataset, 그리고 60,601 개의 Test dataset 으로 구분한 뒤 다음 각 물음에 답하시오. 분류 성능을 
평가/비교할 때는 3-class classification 의 Accuracy 와 Balanced Correction Rate (BCR)을 이용하시오.

In [10]:
# train test split
X_train_L, X_test, y_train_L, y_test = train_test_split(features, target, test_size = 60601, random_state = 42)
# train valid split
X_train, X_valid, y_train, y_valid = train_test_split(X_train_L, y_train_L, test_size = 50000, random_state = 42)

print(len(X_train), len(X_valid), len(X_test))

150000 50000 60601


In [11]:
# performance evaluation function
from sklearn.metrics import confusion_matrix

def perf_eval_fc(y_pred, y_test):
    cm = confusion_matrix(y_test, y_pred)
    #cm 3x3 matrix
    accuracy = np.trace(cm) / np.sum(cm)
    BCR = np.cbrt(cm[0,0]/np.sum(cm[0,:]) * cm[1,1]/np.sum(cm[1,:]) * cm[2,2]/np.sum(cm[2,:]))
    return accuracy, BCR
    
    
perf_table = pd.DataFrame(columns = ['Accuracy', 'BCR'])
perf_table
    

,Accuracy,BCR


# Training

[Q1] 다음과 같이 세 가지 단일 모형에 대하여 분류 모델을 구축하고 Accuracy 와 BCR 관점에서 분류 
정확도를 비교해보시오. CART 와 ANN 의 경우 hyperparameter 후보 값들을 명시하고 Validation 
dataset 을 통해서 최적의 값을 찾아서 Test 에 사용하시오.

- logistic regression과 prepruning의 경우에는 train_L 사용하였음.

## Multinomial Logistic Regression

In [12]:
#import multinomial logistic regression
from sklearn.linear_model import LogisticRegression
import pickle

try:
    #load the model
    logreg = pickle.load(open('logreg_model.sav', 'rb'))
except:
    #train the model
    logreg = LogisticRegression(solver='sag', max_iter=1000, 
                                multi_class='multinomial', random_state=42, verbose=1)

    logreg.fit(X_train_L, y_train_L)

    #save the model

    pickle.dump(logreg, open('logreg_model.sav', 'wb'))

#predict the test set
y_pred = logreg.predict(X_test)

#performance evaluation
perf_table.loc['Logistic Regression'] = perf_eval_fc(y_pred, y_test)
perf_table


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 294 epochs took 88 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min finished


,Accuracy,BCR
Logistic Regression,0.588786,0.372573


## Classification and Regression Tree (CART) 

In [13]:
tree_perf_table = pd.DataFrame(columns = ['Accuracy', 'BCR'])

In [14]:
from sklearn.tree import DecisionTreeClassifier

try:
    #load the model
    full_tree = pickle.load(open('full_tree_model.sav', 'rb'))
    
except:
    full_tree = DecisionTreeClassifier(random_state=42)
    full_tree.fit(X_train, y_train)
    
    #save the model
    pickle.dump(full_tree, open('full_tree_model.sav','wb'))

y_pred = full_tree.predict(X_test)
tree_perf_table.loc['Full Tree'] = perf_eval_fc(y_pred, y_test)

tree_perf_table

,Accuracy,BCR
Full Tree,0.647168,0.596013


- Post pruning 을 시도해 보았지만 예상시간이 11시간이 넘어가서, 불가능했다.

In [15]:
from sklearn.model_selection import GridSearchCV

try:
    #load the model
    best_pre_pruning = pickle.load(open('best_pre_pruning_model.sav', 'rb'))
    
except:
    # Pre Pruning parameters
    pre_pruning_hyperparameters = {
        'max_depth': [2, 4, 8, 16, 32],
        'min_samples_split': [2, 4, 8, 16, 32],
    }  

    # choosing the best hyperparameters using 5-fold cross validation
    pre_pruning = GridSearchCV(DecisionTreeClassifier(random_state=42),
                                scoring='balanced_accuracy',
                                param_grid=pre_pruning_hyperparameters,
                                cv=5,
                                verbose=1,
                                n_jobs=-1)

    pre_pruning.fit(X_train_L, y_train_L)

    best_pre_pruning = pre_pruning.best_estimator_
    
    
tree_perf_table.loc['Pre Pruning'] = perf_eval_fc(best_pre_pruning.predict(X_test), y_test)


Fitting 5 folds for each of 25 candidates, totalling 125 fits


## Artificial Neural Network (ANN) 

In [16]:
neural_network_hyperparameters = {
    'hidden_layer_sizes': [(100,), (100, 100), (100, 100, 100)],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.0001, 0.001, 0.01],
}

from sklearn.neural_network import MLPClassifier

try:
    #load the model
    best_nn = pickle.load(open('best_nn_model.sav', 'rb'))

except:
    nn = GridSearchCV(MLPClassifier(random_state=42, max_iter=1000),
                      scoring='balanced_accuracy',
                      param_grid=neural_network_hyperparameters,
                      cv=5,
                      verbose=1,
                      n_jobs=-1)

    nn.fit(X_train_L, y_train_L)

    best_nn = nn.best_estimator_

    pickle.dump(best_nn, open('best_nn_model.sav', 'wb'))
    
tree_perf_table.loc['Neural Network'] = perf_eval_fc(best_nn.predict(X_test), y_test)

tree_perf_table

Fitting 5 folds for each of 27 candidates, totalling 135 fits
